In [1]:
!pip install selenium
!pip install beautifulsoup4

In [2]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [3]:
def get_jobs(keyword, num_jobs, verbose, path, slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    service = Service(executable_path=path)
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_window_size(1120, 1000)
    
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
#             driver.find_element_by_class_name("selected").click()
            driver.find_element(By.CLASS_NAME, "selected").click()
#             driver.find_element(By.NAME ,"selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
#             driver.find_element_by_css_selector('[alt="Close"]').click() #clicking to the X
            driver.find_element(By.CLASS_NAME, "e1jbctw80 ei0fd8p1 css-1n14mz9 e1q8sty40").click()
            print(' x out worked')
        except NoSuchElementException:
            print(' x out failed')
            pass
        print('go')

        
        #Going through each job in this page
#         job_buttons = driver.find_elements_by_class_name("jl")
        job_buttons = driver.find_element(By.CLASS_NAME, "react-job-listing")
    #jl for Job Listing. These are the buttons we're going to click.
        print('past')
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
#                     company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    company_name = driver.find_element(By.XPATH, './/div[@class="employerName"]').text
#                     location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    location = driver.find_element(By.XPATH, './/div[@class="location"]').text
#                     job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_title = driver.find_element(By.XPATH, './/div[contains(@class, "title")]').text
#                     job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    job_description = driver.find_element(By.XPATH, './/div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
#                 salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
                salary_estimate = driver.find_element(By.XPATH, './/span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
#                 rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
                rating = driver.find_element(By.XPATH, './/span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
#                 driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()
                driver.find_element(By.XPATH, './/div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
#                     headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                    headquarters = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs
            
            
        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)

In [1]:
#This line will open a new chrome window and start the scraping.
path = '/Users/dilshod/Desktop/salary_prediction_project/chromedriver'
df = get_jobs('data scientist',1000, False, path, 15)
df

In [55]:
def get_glassdoor_jobs(keyword, num_jobs):
    from bs4 import BeautifulSoup
    from selenium import webdriver
    import time
    import pandas as pd

    PATH = '/Users/dilshod/Desktop/salary_prediction_project/chromedriver'


    l=list()
    o={}
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="

    target_url = "https://www.glassdoor.com/Job/new-york-python-jobs-SRCH_IL.0,8_IC1132348_KO9,15.htm?clickSource=searchBox"

    options = webdriver.ChromeOptions()
    service = Service(executable_path=PATH)
    driver = webdriver.Chrome(service=service, options=options)

    driver.get(url)

    driver.maximize_window()
    time.sleep(2)

    resp = driver.page_source
    driver.close()

    soup=BeautifulSoup(resp,'html.parser')

    allJobsContainer = soup.find("ul",{"class":"css-7ry9k1"})

    allJobs = allJobsContainer.find_all("li")
    
    while len(l) < num_jobs:
        for job in allJobs:
            try:
                o["Company Name"]=job.find("div",{"class":"job-search-8wag7x"}).text
            except:
                o["Company Name"] = None

            try:
                o["Job Title"]=job.find("div",{"class":"job-title mt-xsm"}).text
            except:
                o["Job Title"] = None


            try:
                o["Location"]=job.find("div",{"class":"location mt-xxsm"}).text
            except:
                o["Location"] = None


            try:
                o["Salary Estimate"]=job.find("div",{"class":"salary-estimate"}).text
            except:
                o["Salary Estimate"] = None

            try:
                job_description = driver.find_element(By.XPATH, './/div[@class="jobDescriptionContent desc"]').text
                print(job_description)
                o['Job Description'] = job_description
            except:
                 o['Job Description'] = None

            l.append(o)

            o={}
#         time.sleep(3)
#         scrolling_element = driver.find_element(By.XPATH, '//*[@id="MainCol"]')
#         driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrolling_element)
#         time.sleep(3)
#         driver.find_element(By.XPATH, '//*[@id="MainCol"]/div[2]/div/div[1]/button[7]').click()
#         time.sleep(3)


    df = pd.DataFrame(l)
    return df

In [2]:
x = get_glassdoor_jobs('data scientist', 1000)

In [45]:
x

,Company Name,Job Title,Location,Salary Estimate,Job Description
0,Logistics Management Institute4.2 ★,Junior Data Scientist/Data Modeler,Remote,None,None
1,CVS Health3.1 ★,Data Scientist - Payment Accuracy,New York State,$70K - $140K (Employer est.),None
2,Juji,Artificial Intelligence (AI) Trainer,"San Jose, CA",$102K - $161K (Glassdoor est.),None
3,Photon3.8 ★,Data Scientist,Washington State,None,None
4,Eccalon LLC2.8 ★,Data Scientist/Database Administrator,"Hanover, MD",$80K - $125K (Glassdoor est.),None
...,...,...,...,...,...
1015,"VMR Strategic Solutions, LLC",Data Scientist,"Bethesda, MD",$100K - $140K (Glassdoor est.),None
1016,Avenue Code4.3 ★,Machine Learning Engineer,United States,$140K - $190K (Employer est.),None
1017,Facebook App3.9 ★,"Data Scientist, Product Analytics - Machine Le...","Burlingame, CA",$165K - $232K (Employer est.),None
1018,Capgemini3.8 ★,Data Base Administration,Remote,None,None
